In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [20]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [21]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training'
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [22]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation'
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [23]:
model = models.Sequential()
model.add(ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(264, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 264)               26493192  
                                                                 
 dropout_2 (Dropout)         (None, 264)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               33920     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/100
20/20 [==============================] - 48s 490ms/step - loss: 0.6195 - accuracy: 0.6750 - val_loss: 0.6664 - val_accuracy: 0.6313
Epoch 2/100
20/20 [==============================] - 9s 377ms/step - loss: 0.3484 - accuracy: 0.8656 - val_loss: 1.3528 - val_accuracy: 0.4938
Epoch 3/100
20/20 [==============================] - 9s 395ms/step - loss: 0.3961 - accuracy: 0.8469 - val_loss: 1.0813 - val_accuracy: 0.4938
Epoch 4/100
20/20 [==============================] - 9s 394ms/step - loss: 0.2674 - accuracy: 0.9094 - val_loss: 1.4820 - val_accuracy: 0.4938
Epoch 5/100
20/20 [==============================] - 9s 390ms/step - loss: 0.2716 - accuracy: 0.9000 - val_loss: 1.8022 - val_accuracy: 0.4938
Epoch 6/100
20/20 [==============================] - 9s 408ms/step - loss: 0.2093 - accuracy: 0.9281 - val_loss: 1.8454 - val_accuracy: 0.4938
Epoch 7/100
20/20 [==============================] - 9s 387ms/step - loss: 0.1444 - accuracy: 0.9578 - val_loss: 1.7382 - val_accuracy: 0.512

In [27]:
model.save('/content/drive/MyDrive/resnet50dropout_indoor_outdoor_model.h5')